# Day 17

In [1]:
from intcode_machine import IntcodeComputer

In [2]:
with open('data/day17.txt') as fn:
    program = fn.readlines()
    program = [int(i) for i in program[0].split(',')]

In [3]:
ASCII = IntcodeComputer(program)

In [4]:
ASCII.run()

10

In [5]:
def draw_ascii(outputs):
    print("".join([chr(x) for x in outputs]))
        

In [6]:
a = draw_ascii(ASCII.outputs)

..............###########.###########................
..............#.........#.#.........#................
..............#.......#########.....#................
..............#.......#.#.#...#.....#................
..............#.......#.#.#...#.....#................
..............#.......#.#.#...#.....#................
..............#.#########.#...#.....#................
..............#.#.....#...#...#.....#................
..............#.#.....#...#########.#................
..............#.#.....#.......#...#.#................
..............#########.......#######................
................#.................#..................
................#...........#########................
................#...........#.....#.#................
................#########...#.....#.#................
........................#...#.....#.#................
........................#...#.....#.#................
........................#...#.......#................
........................#...

In [7]:
import numpy as np

def ascii_to_array(outputs):
    char_array = np.array([chr(x) for x in outputs])
    rowsize = np.where(char_array== '\n')[0][0] + 1
    return char_array[:-1].reshape(-1, rowsize)[:,:-1]


In [8]:
array = ascii_to_array(ASCII.outputs)

In [9]:
intersections = []
for i in range(1,array.shape[0]-1):
    for j in range(1,array.shape[1]-1):
        if all([array[i, j]=='#',
              array[i+1, j]=='#',
              array[i-1, j]=='#',
              array[i, j+1]=='#',
              array[i, j-1]=='#']):
            intersections.append((i,j))

In [10]:
np.sum([np.product(i) for i in intersections])

2660

# Q2

In [11]:
starting_point = np.where(array=='^')

In [12]:
starting_point = [starting_point[0][0], starting_point[1][0]]

In [108]:
direction = 'L'
point = starting_point.copy()

def go(point, turn, facing):
    i = 0
    
    r_map = {'U' : 'R', 'L' : 'U', 'D' : 'L', 'R' :'D'}
    l_map = {v : k for k, v in r_map.items()}
    if turn =='R':
        facing = r_map[facing]
    elif turn =='L':
        facing = l_map[facing]

    while True:
        if facing == 'L':
            newpoint = [point[0], point[1]-1]
        elif facing == 'R':
            newpoint = [point[0], point[1]+1]
        elif facing == 'D':
            newpoint = [point[0]+1, point[1]]
        elif facing == 'U':
            newpoint = [point[0]-1, point[1]]
        
        try:
            if array[newpoint[0], newpoint[1]]!='#':
                break
        except:
            break
        point=newpoint.copy()
        i+=1
    
    #print(point, facing)
    return turn, i, point, facing

In [111]:
directions = []

In [112]:
point = starting_point
facing = 'U'

for direction in ['L', 'R', 'L', 'R',
                  'L', 'L','R', 'L',
                 'R', 'L', 'R' , 'R',
                 'L' , 'L','L','L',
                 'R','L', 'R', 'L',
                 'L', 'R', 'R', 'L',
                 'L', 'L', 'L', 'L',
                 'R','R','L','L',
                 'L','L','L','R']:
    turn, i, point, facing = go(point, turn=direction, facing=facing)
    #assert point!= old_point, 'Err'
    old_point = point
    directions += [turn, i]

In [136]:
def pop_seq(seq, sub_seq):
    seq = seq.copy()
    for i in range(len(seq) - len(sub_seq)+1):
        if seq[i:(i+len(sub_seq))] == sub_seq:
            seq[i:(i+len(sub_seq))] = [None for _ in range(len(sub_seq))]
    return [s for s in seq if s is not None]

In [159]:
min_l = 100
for i in range(0, 22, 2):
    sub_seq = pop_seq(directions, directions[:i])
    for j in range(0,20, 2):
        sub_seq_1 = pop_seq(sub_seq, sub_seq[:j])
        for k in range(0, 18, 2):
            sub_seq_2 = pop_seq(sub_seq_1, sub_seq_1[:k])
            if len(sub_seq_2)==0:
                print(f"{i,j,k}")
                break


(8, 6, 8)
(8, 12, 12)
(8, 14, 6)
(8, 16, 16)
(8, 18, 12)
(14, 16, 14)


In [160]:
A = directions[:8]
sub_seq = pop_seq(directions, A)

In [161]:
B = sub_seq[:6]
sub_seq = pop_seq(sub_seq, B)

In [162]:
C = sub_seq[:8]
sub_seq = pop_seq(sub_seq, C)

In [163]:
A

['L', 10, 'R', 8, 'L', 6, 'R', 6]

In [164]:
B

['L', 8, 'L', 8, 'R', 8]

In [165]:
C

['R', 8, 'L', 6, 'L', 10, 'L', 10]

In [180]:
X = A+B+A+C+A+B+C+B+C+B 
print(X == directions[:(len(X))])

True


## Into the computer

In [183]:
main_routine = ['A', 'B', 'A', 'C', 'A', 'B', 'C', 'B', 'C', 'B']

In [213]:
program2 = program.copy()
program2[0] = 2

In [236]:
def make_code(inp):
    code = [[ord(str(i)[0]),44] if len(str(i))==1 else [ord(str(i)[0]),ord(str(i)[1]),44] for i in inp]
    code = [d for c in code for d in c]
    code[-1] = 10
    return code

In [248]:
code1 = make_code(main_routine)
code2 = make_code(A)
code3 = make_code(B)
code4 = make_code(C)
code5 = make_code(['n'])

In [256]:
ASCII2 = IntcodeComputer(program2)

In [257]:
ASCII2.run(code1)
ASCII2.run(code2)
ASCII2.run(code3)
ASCII2.run(code4)
ASCII2.run(code5)

790595